In [1]:
#imports
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service as ChromeService 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.selenium_manager import SeleniumManager 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from time import sleep
from googletrans import Translator
from google_trans_new import google_translator  
from deep_translator import GoogleTranslator
import os
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import pandas as pd
from sqlalchemy import create_engine
import warnings
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType, StringType
import pyspark.pandas as ps
warnings.filterwarnings('ignore')

c:\Users\ayush\anaconda3\lib\site-packages\pyspark\pandas\__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
#Functions
def translator(s,source):
    return GoogleTranslator(source=source, target='en').translate(s)


def translator_v2(s,source='auto'):
        translator = Translator()
        return translator.translate(s, src='hi',dest='en').text if s != '' else 'null' 

translate_udf = udf(translator_v2,StringType())

In [4]:
#selenium webdriver
input_params_dropdown_by_id = {}
input_params_dropdown_by_id['dbselect'] = '2023' #Select Year
input_params_dropdown_by_id['district_id'] = 'मुंबई उपनगर' #District
input_params_dropdown_by_id['taluka_id'] = 'अंधेरी' #Taluka
input_params_dropdown_by_id['village_id'] = 'बांद्रा' #Village

input_params_text_by_id = {}
input_params_text_by_id['free_text'] = '2023' 

url = 'https://pay2igr.igrmaharashtra.gov.in/eDisplay/Propertydetails/index' 
driver = webdriver.Chrome() 
driver.implicitly_wait(10)
driver.get(url)

for key, value in input_params_dropdown_by_id.items():
	print(key,value)
	dropdown = driver.find_element(by=By.ID,value=key)
	dropdown_select = Select(dropdown)
	for option in dropdown_select.options:
		if option.text == value:
			option.click()
			print(f'Clicked {option.text} for {key}')
			break
	driver.implicitly_wait(2)

#Input reg year
driver.find_element(by=By.ID, value='free_text').send_keys(input_params_text_by_id['free_text'])
driver.implicitly_wait(2)

#Input Captcha
captcha = input()
driver.find_element(by=By.ID, value='cpatchaTextBox').send_keys(captcha)
print(f'Entered Captcha {captcha}')
driver.implicitly_wait(2)

#Submit to get result set
driver.find_element(by=By.ID,value='submit').click()
driver.implicitly_wait(10)


#Click on 50 Pages
dropdown = driver.find_element(by=By.NAME, value='tableparty_length')
dropdown_select = Select(dropdown)
for option in dropdown_select.options:
	if option.text == 'All':
		option.click()
		print('Set to All Records')
		break

print('So Far, So Good')

dbselect 2023
Clicked 2023 for dbselect
district_id मुंबई उपनगर
Clicked मुंबई उपनगर for district_id
taluka_id अंधेरी
Clicked अंधेरी for taluka_id
village_id बांद्रा
Clicked बांद्रा for village_id
Entered Captcha JCIW9D
Set to All Records
So Far, So Good


In [21]:
#scrap records
records_raw = pd.DataFrame(columns=['अनु क्र.','दस्त क्र.','दस्त प्रकार','दू. नि. कार्यालय','वर्ष','लिहून देणार','लिहून घेणार','इतर माहीती','सूची क्र. २'])

for index, table in enumerate(driver.find_elements(by=By.ID, value='tbdata')):
    data = [item.text if item.text != 'सूची क्र. २' else item.find_element(by=By.TAG_NAME,value='a').get_attribute('href') for item in table.find_elements(by=By.XPATH, value=".//*[self::td or self::th]")]
    records_raw.loc[len(records_raw)] = data
    # print(data)
    print(f'{index} Row appended')

records_raw
records_translate = records_raw.copy()

NameError: name 'driver' is not defined

In [6]:
# Insert Scraped Raw Data in postgres
db_user = 'postgres'
db_password = 'Sunrise12345'
db_host = '127.0.0.1'
db_port = '5432'
db_name = 'propReturns'

engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')
table_name = 'record_details_raw'

records_raw.to_sql(table_name, engine, if_exists='replace', index=True)

print(f"{len(records_raw)} DataFrame has been inserted into the '{table_name}' table.")

1209 DataFrame has been inserted into the 'record_details_raw' table.


In [8]:
#Approach Via Pandas --> Extremely Slow
records_translate = records_raw.copy()
records_translate.columns = [translator(col,source='auto') for col in records_translate.columns]
cols_to_translate = [
    'diarrhea type',
    'Du. Prohibit. Office',
    'Will write', 
    'Will write down', 
    'Other information'
    ]

records_translated = pd.DataFrame(columns=records_translate.columns)
batch_size = 100
start = 0
for batch in range(start,len(records_translate),batch_size):
    print(f'Batch -> [{batch},{batch+batch_size}]')
    temp_df = records_translate.iloc[batch:batch+batch_size,:].copy()
    for col in cols_to_translate:
        print(f'Translating {col}')
        temp_df[col] = temp_df[col].apply(lambda x: translator(x,source='hi'))
        print(f'Done')
    records_translated = pd.concat([records_translated,temp_df])
records_translated.rename(columns={'Will write':'buyer_name','Will write down':'seller_name'},inplace=True)

Batch -> [0,100]
Translating diarrhea type
Done
Translating Du. Prohibit. Office
Done
Translating Will write


RequestError: Request exception can happen due to an api connection error. Please check your connection and try again

In [3]:
# del os.environ['PYSPARK_PYTHON'] 
# del os.environ['PYSPARK_DRIVER_PYTHON'] 
os.environ['PYSPARK_PYTHON'] = sys.executable 
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [4]:
spark.stop()
del spark

NameError: name 'spark' is not defined

In [5]:
# Approach Via Spark
spark = SparkSession.builder.getOrCreate()
# spark = SparkSession.builder \
#         .config('spark.executor.instances', 4) \
#         .config("spark.sql.execution.arrow.enabled", "true") \
#         .config("spark.dynamicAllocation.enabled", "true") \
#         .config("spark.dynamicAllocation.minExecutors","1") \
#         .config("spark.dynamicAllocation.maxExecutors","5") \
#         .config("spark.executor.cores",6) \
#         .config("spark.executor.memory","2g") \
#         .getOrCreate()
# spark.conf.set("spark.sql.execution.arrow.enabled", "true")
# #spark.conf.set("spark.dynamicAllocation.enabled", "true")
# spark.conf.set("spark.executor.cores", 4)
# spark.conf.set("spark.dynamicAllocation.minExecutors","1")
# spark.conf.set("spark.dynamicAllocation.maxExecutors","5")
# conf = spark.sparkContext.getConf()
spark

In [6]:
#Get the Data from SQL directly
host = '127.0.0.1'
port = '5432'
database = 'propReturns'
username = 'postgres'
password = 'Sunrise12345'
url = f"jdbc:postgresql://{host}:{port}/{database}"
table_name = 'record_details_raw'

engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database}')

spark_df = spark.read.format("jdbc") \
        .option("url", url) \
        .option("driver", "org.postgresql.Driver") \
        .option("dbtable", table_name) \
        .option("user", username) \
        .option("password", password).load()

records_raw = pd.read_sql('record_details_raw',engine)

In [7]:
new_names = []
for col in records_raw.columns:
    new_names.append(translator(col,source='auto'))

new_names = [x.replace('.','') for x in new_names]
spark_df = spark_df.toDF(*new_names)

In [8]:
spark_df.show()

+-----+-----+-----------+--------------------+------------------+----------+--------------------+--------------------+--------------------+--------------------+
|index|Sl no|Diarrhea no|       diarrhea type|Du Prohibit Office|      Year|          Will write|     Will write down|   Other information|           List no 2|
+-----+-----+-----------+--------------------+------------------+----------+--------------------+--------------------+--------------------+--------------------+
|    0|    1|       4286|       विकसनकरारनामा|सह दु.नि. अंधेरी 7|10/03/2023|1) अजित सिंह करता...|1) नाविश रियल्टी ...|1) इतर माहिती: जम...|https://pay2igr.i...|
|    1|    2|      13217|           भाडेपट्टा|सह दु.नि. अंधेरी 7|25/07/2023|1) श्रीमती चंद्रक...|1) महाराष्ट़ हाऊस...|1) इतर माहिती: पि...|https://pay2igr.i...|
|    2|    3|       2449|             सेल डीड|सह दु.नि. अंधेरी 7|09/02/2023|1) नूपुर अनिल कळक...|1) हर्ष सतपाल मल्...|1) इतर माहिती: सद...|https://pay2igr.i...|
|    3|    4|       8691|66-नोटीस 

In [9]:
cols_to_translate = [
    'diarrhea type',
    'Du Prohibit Office',
    'Will write', 
    'Will write down', 
    'Other information'
    ]

for column in cols_to_translate:
    spark_df = spark_df.withColumn(column, translate_udf(spark_df[column]))

spark_df = spark_df.withColumnRenamed(existing='Will write',new='buyer_name')
spark_df = spark_df.withColumnRenamed(existing='Will write down',new='seller_name')

In [10]:
spark_df.show()

+-----+-----+-----------+--------------------+------------------+----------+--------------------+--------------------+--------------------+--------------------+
|index|Sl no|Diarrhea no|       diarrhea type|Du Prohibit Office|      Year|          buyer_name|         seller_name|   Other information|           List no 2|
+-----+-----+-----------+--------------------+------------------+----------+--------------------+--------------------+--------------------+--------------------+
|    0|    1|       4286|development agree...|Co. D.N. Andheri 7|10/03/2023|1) Ajit Singh Kar...|1) Paresh Ranchho...|1) Other informat...|https://pay2igr.i...|
|    1|    2|      13217|               lease|Co. D.N. Andheri 7|25/07/2023|1) Bhavin Sheth, ...|1) Hanmant Dhanva...|1) Other informat...|https://pay2igr.i...|
|    2|    3|       2449|           sale deed|Co. D.N. Andheri 7|09/02/2023|1) Nupur Anil Kal...|1) Harsh Satpal M...|1) Other Informat...|https://pay2igr.i...|
|    3|    4|       8691|66-Notice

In [39]:
table_name_sink = 'record_details_translated'

spark_df.select("*").write.format("jdbc")\
    .option("url", url) \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", table_name_sink) \
    .option("user", username) \
    .option("password", password).save()

In [11]:
records_translated = spark_df.toPandas()

In [10]:
records_translated = spark_df.toPandas()

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "C:\Users\ayush\AppData\Local\Temp\ipykernel_16296\3257740425.py", line 8, in translator_v2
  File "c:\Users\ayush\anaconda3\lib\site-packages\googletrans\client.py", line 210, in translate
    data, response = self._translate(text, dest, src, kwargs)
  File "c:\Users\ayush\anaconda3\lib\site-packages\googletrans\client.py", line 108, in _translate
    r = self.client.get(url, params=params)
  File "c:\Users\ayush\anaconda3\lib\site-packages\httpx\_client.py", line 755, in get
    return self.request(
  File "c:\Users\ayush\anaconda3\lib\site-packages\httpx\_client.py", line 600, in request
    return self.send(
  File "c:\Users\ayush\anaconda3\lib\site-packages\httpx\_client.py", line 620, in send
    response = self.send_handling_redirects(
  File "c:\Users\ayush\anaconda3\lib\site-packages\httpx\_client.py", line 647, in send_handling_redire

In [55]:
records_translated = spark_df.toPandas()
table_name = 'record_details_translated'
records_translated.to_sql(table_name, engine, if_exists='replace', index=True)
print(f"{len(records_translate)} DataFrame has been inserted into the '{table_name}' table.")

1209 DataFrame has been inserted into the 'record_details_translated' table.
